In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

In [3]:
import pandas as pd
import math
import seaborn as sns
from matplotlib import pyplot as plt
import re
import numpy as np
import geopandas as gpd
from shapely.geometry import Point # Point class
from shapely.geometry import shape
import io
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
from geopy.geocoders import Nominatim
from time import sleep
from datetime import datetime

C:\Users\mmmiah\AppData\Local\ESRI\conda\envs\geo_env\Lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\mmmiah\AppData\Local\Temp\ipykernel_1688\2960822959.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https

In [36]:
madt=pd.read_csv('D:/AT Census/permanent_counter_2018_2022_MADT_Full_month_data.csv')
madt["year"]=madt["year"].astype(np.int64)
madt["month"]=madt["month"].astype(np.int64)
madt.head()

,Unnamed: 0,ID,location,latitude,longitude,year,month,MTT,days_in_month,Simple_MADT,AASHTO_MADT,diff,type,geometry
0,0,19211,urban,32.59,-117.11,2018,1,5277.00,31,170.23,171.53,-1.30,eco_permanent,POINT (-117.11242 32.589455)
1,1,19211,urban,32.59,-117.11,2018,2,26993.00,28,964.04,964.04,0.00,eco_permanent,POINT (-117.11242 32.589455)
2,2,19211,urban,32.59,-117.11,2018,3,13817.00,31,445.71,454.36,-8.65,eco_permanent,POINT (-117.11242 32.589455)
3,3,19211,urban,32.59,-117.11,2018,4,6368.00,30,212.27,212.74,-0.48,eco_permanent,POINT (-117.11242 32.589455)
4,4,19211,urban,32.59,-117.11,2018,5,6150.00,31,198.39,198.98,-0.59,eco_permanent,POINT (-117.11242 32.589455)


In [5]:
loc=madt[['ID','year','latitude','longitude','location']]
loc1=loc.drop_duplicates(subset=['ID','latitude','longitude'], keep='first')
loc1.shape

(208, 5)

In [6]:
# bring the location based on lat and long
def city_state_country(row):
    coord = f"{row['latitude']}, {row['longitude']}"
    sleep(1)
    location = geolocator.reverse(coord, exactly_one=True, language='en')
    if not location:
        # if you see many in a row, it's probably Nominatim blocking you.
        # if it's just every once in a while, there were just some bad results. 
        print('Failed with coord: ', coord)
        row['road'],row['city'],row['county'], row['state'], row['country'] = None, None, None
        return row
    address = location.raw['address']
    city = address.get('city', '')
    county=address.get('county', '')
    state = address.get('state', '')
    country = address.get('country', '')
    row['road']=address.get('road', '')
    row['city'] = city
    row['county'] = county
    row['state'] = state
    row['country'] = country
    return row

In [7]:
loc2 = loc1.apply(city_state_country, axis=1)
loc2.head()

,ID,year,latitude,longitude,location,road,city,county,state,country
0,19211,2018,32.59,-117.11,urban,Bayshore Bikeway,,,California,United States
30,14096,2018,32.59,-117.12,urban,Delaware Street,,San Diego County,California,United States
60,13761,2018,32.64,-117.10,urban,Bayshore Bikeway,Chula Vista,San Diego County,California,United States
100,3576,2018,32.65,-117.10,urban,Bayshore Bikeway,,San Diego County,California,United States
140,19210,2018,32.66,-117.07,urban,Sweetwater Bikeway,Chula Vista,San Diego County,California,United States


In [8]:
loc3=loc2.drop(['year'],axis=1)

In [ ]:
# madt1=pd.merge(madt, loc3, on=['ID',"latitude",'longitude'], how='inner')
# madt1.head()

In [9]:
# check=madt1.drop_duplicates(subset=['ID'], keep='first')
# check.shape

(208, 19)

In [9]:
loc3.shape

(208, 9)

In [10]:
# save the data
loc3.to_csv('D:/AT Census/Ped_Unique_counter_2018_2022_location.csv')

# Now Bring the pedestrian Facilities, Strava and Merge with Eco Counters

In [12]:
# Import ped Facilities
ped_fac=gpd.read_file('D:/AT Census/Eco_Facilities_feb8.shp')
ped_fac.head()

,Join_Count,TARGET_FID,Field1,ID,latitude,longitude,location,road,city,county,...,length,City_Bicyc,Comments1,Comments2,STATEFP,COUNTYFP,COUNTYNS,GEOID,Location_C,geometry
0,1,1,0,19211,32.59,-117.11,urban,Bayshore Bikeway,None,None,...,541.15,Path,None,None,6,73,277301,6073,San Diego,POINT (-117.11242 32.58946)
1,1,2,30,14096,32.59,-117.12,urban,Delaware Street,None,San Diego County,...,55.58,Path,None,City Bicycle Facilities Assigned,6,73,277301,6073,San Diego,POINT (-117.11634 32.59009)
2,1,3,60,13761,32.64,-117.10,urban,Bayshore Bikeway,Chula Vista,San Diego County,...,1485.00,Path,None,None,6,73,277301,6073,San Diego,POINT (-117.10147 32.64154)
3,1,4,100,3576,32.65,-117.10,urban,Bayshore Bikeway,None,San Diego County,...,1485.00,Path,None,None,6,73,277301,6073,San Diego,POINT (-117.10370 32.64904)
4,1,5,140,19210,32.66,-117.07,urban,Sweetwater Bikeway,Chula Vista,San Diego County,...,1421.96,Path,None,None,6,73,277301,6073,San Diego,POINT (-117.07013 32.65724)


In [15]:
ped_fac.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [13]:
ped_fac.columns

Index(['Join_Count', 'TARGET_FID', 'Field1', 'ID', 'latitude', 'longitude',
       'location', 'road', 'city', 'county', 'state', 'country', 'serial_num',
       'user_type', 'domain', 'site_name', 'lat', 'long', 'osmid', 'name',
       'OSM_Functi', 'oneway', 'reversed', 'length', 'City_Bicyc', 'Comments1',
       'Comments2', 'STATEFP', 'COUNTYFP', 'COUNTYNS', 'GEOID', 'Location_C',
       'geometry'],
      dtype='object')

In [16]:
ped_fac1=ped_fac[[ 'ID', 'latitude', 'longitude','location', 'road', 'city', 'county','City_Bicyc','OSM_Functi','osmid','geometry']]
ped_fac1.head()

,ID,latitude,longitude,location,road,city,county,City_Bicyc,OSM_Functi,osmid,geometry
0,19211,32.59,-117.11,urban,Bayshore Bikeway,None,None,Path,cycleway,6007743,POINT (-117.11242 32.58946)
1,14096,32.59,-117.12,urban,Delaware Street,None,San Diego County,Path,residential,6050490,POINT (-117.11634 32.59009)
2,13761,32.64,-117.10,urban,Bayshore Bikeway,Chula Vista,San Diego County,Path,cycleway,"[232486561, 554351624, 240067888, 232484016, 2...",POINT (-117.10147 32.64154)
3,3576,32.65,-117.10,urban,Bayshore Bikeway,None,San Diego County,Path,cycleway,"[232486561, 554351624, 240067888, 232484016, 2...",POINT (-117.10370 32.64904)
4,19210,32.66,-117.07,urban,Sweetwater Bikeway,Chula Vista,San Diego County,Path,cycleway,"[503430718, 234828855]",POINT (-117.07013 32.65724)


In [19]:
# Read Strava Data
stv=pd.read_csv('D:/AT Census/eco_strava_monthly_matching2018_2022.csv')
stv.head()

,Unnamed: 0,ID,latitude,longitude,edgeUID,osmId,dist,edge_uid,activity_type,month,...,reverse_65_plus_people_count,forward_average_speed,reverse_average_speed,osm_reference_id,year,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio
0,0,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,"Run,Walk,Hike",2018-08,...,0,0.00,2.24,773713312,2018,5,0,5,0.00,1.00
1,1,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,"Run,Walk,Hike",2018-01,...,0,0.00,1.50,773713312,2018,5,0,10,0.00,2.00
2,2,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,"Run,Walk,Hike",2018-04,...,0,0.00,2.29,773713312,2018,5,0,20,0.00,4.00
3,3,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,"Run,Walk,Hike",2018-07,...,0,1.97,0.00,773713312,2018,5,0,5,0.00,1.00
4,4,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,"Run,Walk,Hike",2018-03,...,0,2.14,1.71,773713312,2018,10,0,10,0.00,1.00


In [20]:
stv1=stv[['ID', 'latitude', 'longitude', 'edgeUID', 'osmId',
       'dist', 'edge_uid', 'osm_reference_id',  'month',
       'stv_total', 'stv_com_total',
       'stv_rec_total', 'stv_com_ratio', 'stv_rec_ratio']]

In [21]:
stv1.head()

,ID,latitude,longitude,edgeUID,osmId,dist,edge_uid,osm_reference_id,month,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio
0,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,773713312,2018-08,5,0,5,0.00,1.00
1,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,773713312,2018-01,5,0,10,0.00,2.00
2,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,773713312,2018-04,5,0,20,0.00,4.00
3,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,773713312,2018-07,5,0,5,0.00,1.00
4,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,773713312,2018-03,10,0,10,0.00,1.00


In [23]:
stv1[[ "year","month"]] = stv1["month"].str.split("-", expand = True)
stv1.head()

C:\Users\mmmiah\AppData\Local\Temp\ipykernel_1688\1190064131.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stv1[[ "year","month"]] = stv1["month"].str.split("-", expand = True)


,ID,latitude,longitude,edgeUID,osmId,dist,edge_uid,osm_reference_id,month,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,year
0,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,773713312,08,5,0,5,0.00,1.00,2018
1,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,773713312,01,5,0,10,0.00,2.00,2018
2,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,773713312,04,5,0,20,0.00,4.00,2018
3,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,773713312,07,5,0,5,0.00,1.00,2018
4,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,773713312,03,10,0,10,0.00,1.00,2018


In [25]:
stv1['year'].unique()

array(['2018', '2019', '2020', '2021', '2022'], dtype=object)

In [31]:
stv1["year"]=stv1["year"].astype(np.int64)
stv1["month"]=stv1["month"].astype(np.int64)
stv1.head()

C:\Users\mmmiah\AppData\Local\Temp\ipykernel_1688\522184798.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stv1["year"]=stv1["year"].astype(np.int64)
C:\Users\mmmiah\AppData\Local\Temp\ipykernel_1688\522184798.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stv1["month"]=stv1["month"].astype(np.int64)


,ID,latitude,longitude,edgeUID,osmId,dist,edge_uid,osm_reference_id,month,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,year
0,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,773713312,8,5,0,5,0.00,1.00,2018
1,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,773713312,1,5,0,10,0.00,2.00,2018
2,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,773713312,4,5,0,20,0.00,4.00,2018
3,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,773713312,7,5,0,5,0.00,1.00,2018
4,3577,32.59,-117.11,305679030.00,773713312.00,d11,305679030,773713312,3,10,0,10,0.00,1.00,2018


In [33]:
stv1.columns

Index(['ID', 'latitude', 'longitude', 'edgeUID', 'osmId', 'dist', 'edge_uid',
       'osm_reference_id', 'month', 'stv_total', 'stv_com_total',
       'stv_rec_total', 'stv_com_ratio', 'stv_rec_ratio', 'year'],
      dtype='object')

In [38]:
madt.columns

Index(['Unnamed: 0', 'ID', 'location', 'latitude', 'longitude', 'year',
       'month', 'MTT', 'days_in_month', 'Simple_MADT', 'AASHTO_MADT', 'diff',
       'type', 'geometry'],
      dtype='object')

In [51]:
eco_strava=pd.merge( madt, stv1,left_on=['ID','year','month'], right_on=['ID','year','month'], how='inner')
eco_strava['Strava_MADT']=eco_strava['stv_total']/eco_strava['days_in_month']
eco_strava.head()

,Unnamed: 0,ID,location,latitude_x,longitude_x,year,month,MTT,days_in_month,Simple_MADT,...,osmId,dist,edge_uid,osm_reference_id,stv_total,stv_com_total,stv_rec_total,stv_com_ratio,stv_rec_ratio,Strava_MADT
0,0,19211,urban,32.59,-117.11,2018,1,5277.00,31,170.23,...,6007743.00,d11,305679020,6007743,225,10,215,0.04,0.96,7.26
1,1,19211,urban,32.59,-117.11,2018,2,26993.00,28,964.04,...,6007743.00,d11,305679020,6007743,135,10,125,0.07,0.93,4.82
2,2,19211,urban,32.59,-117.11,2018,3,13817.00,31,445.71,...,6007743.00,d11,305679020,6007743,235,5,230,0.02,0.98,7.58
3,3,19211,urban,32.59,-117.11,2018,4,6368.00,30,212.27,...,6007743.00,d11,305679020,6007743,200,15,185,0.07,0.93,6.67
4,4,19211,urban,32.59,-117.11,2018,5,6150.00,31,198.39,...,6007743.00,d11,305679020,6007743,205,15,190,0.07,0.93,6.61


In [52]:
madt.shape, stv1.shape,eco_strava.shape

((3922, 14), (14659, 15), (3473, 27))

In [53]:
# Now join with facilities
eco_strava_fac=pd.merge(eco_strava,ped_fac1, on=['ID'],how='inner')
eco_strava_fac.shape

(3473, 37)

In [54]:
eco_strava_fac.to_csv('D:/AT Census/ped_processed_data/Ped_modeling_data.csv')

In [22]:
# end of the script